In [3]:
#importing required libraries

In [224]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV,RandomizedSearchCV, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from statistics import mean

## train_test_split

In [191]:
df = pd.read_csv('cleaned_data.csv')

In [193]:
df.head()

,Unnamed: 0,bt,km,transmission,ownerNo,oem,model,modelYear,price_in_lakhs,Insurance Validity,Fuel Type,Seats,Safety_count,top_features_count,Color,No of Cylinder,Turbo Charger,Super Charger,City,Power
0,0,Hatchback,120000,Manual,3,Maruti,Maruti Celerio,2015,4.00,third party insurance,Petrol,5,13,8,White,3.0,no,no,Bangalore,67.04
1,1,SUV,32706,Manual,2,Ford,Ford Ecosport,2018,8.11,comprehensive,Petrol,5,27,9,White,3.0,no,no,Bangalore,121.31
2,2,Hatchback,11949,Manual,1,Tata,Tata Tiago,2018,5.85,comprehensive,Petrol,5,24,9,Red,3.0,no,no,Bangalore,84.00
3,3,Sedan,17794,Manual,1,Hyundai,Hyundai Xcent,2014,4.62,comprehensive,Petrol,5,18,9,Others,4.0,no,no,Bangalore,81.86
4,4,SUV,60000,Manual,1,Maruti,Maruti SX4 S Cross,2015,7.90,third party insurance,Diesel,5,22,9,Gray,4.0,yes,no,Bangalore,88.50


In [207]:
df.columns

Index(['bt', 'km', 'transmission', 'ownerNo', 'oem', 'model', 'modelYear',
       'price_in_lakhs', 'Insurance Validity', 'Fuel Type', 'Seats',
       'Safety_count', 'top_features_count', 'Color', 'No of Cylinder',
       'Turbo Charger', 'Super Charger', 'City', 'Power'],
      dtype='object')

In [209]:
df.shape

(8276, 19)

In [197]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
# seperating value and target

In [211]:
val = df.drop('price_in_lakhs',axis=1)
tar = df['price_in_lakhs']

In [54]:
# Standardization

In [203]:
# standardizing numerical data
scaler = StandardScaler()
numeric_columns = val.select_dtypes(include=[np.number]).columns.tolist()
val[numeric_columns] = scaler.fit_transform(val[numeric_columns])
val.head()

,bt,km,transmission,ownerNo,oem,model,modelYear,Insurance Validity,Fuel Type,Seats,Safety_count,top_features_count,Color,No of Cylinder,Turbo Charger,Super Charger,City,Power
0,Hatchback,0.822242,Manual,2.580783,Maruti,Maruti Celerio,-0.408549,third party insurance,Petrol,-0.307065,-1.421235,-1.394561,White,-1.431242,no,no,Bangalore,-0.822502
1,SUV,-0.352303,Manual,1.010531,Ford,Ford Ecosport,0.378009,comprehensive,Petrol,-0.307065,0.566236,0.261067,White,-1.431242,no,no,Bangalore,0.349576
2,Hatchback,-0.631589,Manual,-0.559720,Tata,Tata Tiago,0.378009,comprehensive,Petrol,-0.307065,0.140350,0.261067,Red,-1.431242,no,no,Bangalore,-0.456214
3,Sedan,-0.552945,Manual,-0.559720,Hyundai,Hyundai Xcent,-0.670735,comprehensive,Petrol,-0.307065,-0.711424,0.261067,Others,0.360463,no,no,Bangalore,-0.502432
4,SUV,0.014939,Manual,-0.559720,Maruti,Maruti SX4 S Cross,-0.408549,third party insurance,Diesel,-0.307065,-0.143575,0.261067,Gray,0.360463,yes,no,Bangalore,-0.359027


In [205]:
# onehot encoding all categorical features
encoder = OneHotEncoder(sparse_output=False)
category_columns = val.select_dtypes(include=['object']).columns.tolist()
encoded_features = encoder.fit_transform(val[category_columns])
Df_encoded = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(category_columns))

Df_encoded.head()

,bt_Convertibles,bt_Coupe,bt_Hatchback,bt_Hybrids,bt_MUV,bt_Minivans,bt_Pickup Trucks,bt_SUV,bt_Sedan,bt_Wagon,...,Turbo Charger_twin,Turbo Charger_yes,Super Charger_no,Super Charger_yes,City_Bangalore,City_Chennai,City_Delhi,City_Hyderabad,City_Jaipur,City_Kolkata
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [213]:
# concat encoded to Df_X
VAL = pd.concat([val.drop(category_columns, axis=1), Df_encoded], axis=1)
VAL.head()

,km,ownerNo,modelYear,Seats,Safety_count,top_features_count,No of Cylinder,Power,bt_Convertibles,bt_Coupe,...,Turbo Charger_twin,Turbo Charger_yes,Super Charger_no,Super Charger_yes,City_Bangalore,City_Chennai,City_Delhi,City_Hyderabad,City_Jaipur,City_Kolkata
0,120000,3,2015,5,13,8,3.0,67.04,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,32706,2,2018,5,27,9,3.0,121.31,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,11949,1,2018,5,24,9,3.0,84.00,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,17794,1,2014,5,18,9,4.0,81.86,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,60000,1,2015,5,22,9,4.0,88.50,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


## Model Build and Evaluation

In [215]:
train_data,test_data,train_lab,test_lab = train_test_split(VAL,tar,random_state=56,test_size=0.2)

In [183]:
train_data.shape

(6620, 536)

In [185]:
test_data.shape

(1656, 536)

In [217]:
# list of all models with thier name
models = []
models.append(('Linear Regression', LinearRegression()))
models.append(('Decision Tree Regression', DecisionTreeRegressor()))
models.append(('Random Forest Regression', RandomForestRegressor()))
models.append(('Gradient Boosting Regression', GradientBoostingRegressor()))
# save the neg MSE in results
results = []
for name, model in models:
  kFold = KFold(n_splits=5, shuffle=True, random_state=42)
  cv_results = cross_val_score(model,train_data,train_lab, cv=kFold, scoring='neg_mean_squared_error')
  print(f'{name}: {cv_results.mean()}')
  results.append((name, cv_results.mean()))


Linear Regression: -70046561000.37521
Decision Tree Regression: -97.79992393924137
Random Forest Regression: -54.70365899337992
Gradient Boosting Regression: -57.99203294557212


In [219]:
# let's evaluate all the model using R-square score
for name, model in models:
  model.fit(train_data,train_lab)
  print(f'{name} Training Score: {model.score(train_data,train_lab)}')
  print(f'{name} Testing Score: {model.score(test_data,test_lab)}')

Linear Regression Training Score: 0.6985773056650509
Linear Regression Testing Score: -29735314.604771666
Decision Tree Regression Training Score: 0.9960706969848518
Decision Tree Regression Testing Score: 0.461333886833584
Random Forest Regression Training Score: 0.9532615817457525
Random Forest Regression Testing Score: 0.659869577809058
Gradient Boosting Regression Training Score: 0.7761676679648583
Gradient Boosting Regression Testing Score: 0.6065563020235725


## L2 Regularization

In [228]:
# Ridge (L2) Regression
cross_val_scores_ridge = []
alpha_values = []

# finds the best alpha value for the model 
for i in range(1, 9):
    ridgeModel = Ridge(alpha=i * 0.25)
    ridgeModel.fit(train_data,train_lab)
    scores = cross_val_score(ridgeModel,VAL,tar, cv=10)
    avg_cross_val_score = mean(scores) * 100
    cross_val_scores_ridge.append(avg_cross_val_score)
    alpha_values.append(i * 0.25)

In [230]:
best_alpha_ridge = alpha_values[np.argmax(cross_val_scores_ridge)]
ridgeModelChosen = Ridge(alpha=best_alpha_ridge)
ridgeModelChosen.fit(train_data,train_lab)
print("Ridge Regression Score:", ridgeModelChosen.score(test_data,test_lab))

Ridge Regression Score: 0.5420175896145256


In [232]:
# Lasso (L1) Regression
cross_val_scores_lasso = []
lambda_values = []

# find the best lambda for lasso
for i in range(1, 9):
    lassoModel = Lasso(alpha=i * 0.25, tol=0.0925)
    lassoModel.fit(train_data,train_lab)
    scores = cross_val_score(lassoModel,VAL,tar, cv=10)
    avg_cross_val_score = mean(scores) * 100
    cross_val_scores_lasso.append(avg_cross_val_score)
    lambda_values.append(i * 0.25)

In [236]:
best_lambda_lasso = lambda_values[np.argmax(cross_val_scores_lasso)]
lassoModelChosen = Lasso(alpha=best_lambda_lasso, tol=0.0925)
lassoModelChosen.fit(train_data,train_lab)
print("Lasso Regression Score:", lassoModelChosen.score(test_data,test_lab))

Lasso Regression Score: 0.4067157558344272


In [238]:
# let's train the model using random forest regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(train_data,train_lab)
print("Random Forest Regression Training Score:", rf.score(train_data,train_lab))
print("Random Forest Regression Score:", rf.score(test_data,test_lab))

Random Forest Regression Training Score: 0.9516349475303607
Random Forest Regression Score: 0.6505337574027898


## Hyperparameter Tuning

In [243]:
random_params = {'n_estimators': [100, 200, 300, 400, 600],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [3, 4, None],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

rscv = RandomizedSearchCV(rf, random_params, cv=5, n_jobs=-1, verbose=2, random_state=42)
rscv.fit(train_data,train_lab)
print(f'randomSeachCV Best Score: {rscv.best_score_}')

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\rupsh\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
15 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\rupsh\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\rupsh\anaconda3\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\rupsh\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\rupsh\anaconda3\Lib\site-packages\

randomSeachCV Best Score: 0.6671704423596024


In [245]:
rf_pred = rf.predict(test_data)
print(f'Random Forest Regression MSE: {mean_squared_error(test_lab, rf_pred)}')
print(f'Random Forest Regression R2 Score: {r2_score(test_lab, rf_pred)}')
print(f'Random Forest Regression MAE: {mean_absolute_error(test_lab, rf_pred)}')

Random Forest Regression MSE: 45.94723375502137
Random Forest Regression R2 Score: 0.6505337574027898
Random Forest Regression MAE: 1.9811003324850471
